# PDE-Constrained Shape Optimization in NGSolve

We want to solve the PDE-constrained shape optimization problem
\begin{equation}
            \underset{\Omega\subset \mathsf{D}}{\mbox{min}} \; J(u) := \int_\Omega |u-u_d|^q \; dx, \quad q\ge 2
\end{equation}
 subject to that $(\Omega,u)$ satisfy
 \begin{equation}
           \int_\Omega \nabla u \cdot \nabla v \; dx = \int_\Omega f v \; dx \; \quad \text{ for all } v \in H_0^1(\Omega),
\end{equation}
where $\Omega \subset \mathbb R^2$ for given $u_d, f \in C^1(\mathbb R^2)$.

In [ ]:
import netgen.gui

from ngsolve import *
from netgen.geom2d import SplineGeometry

In [ ]:
geo = SplineGeometry()
geo.AddCircle(c=(0.5,0.5), r=0.5, bc = 'circle')
mesh = Mesh(geo.GenerateMesh(maxh = 0.08))

In [ ]:
#given data of our problem (chosen such that \Omega^* = [0,1]^2 is the optimal shape)
f = CoefficientFunction(2*y*(1-y)+2*x*(1-x))
ud = x*(1-x)*y*(1-y)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

### State equation

In [ ]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
Draw (gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

fstate = LinearForm(fes)
fstate += f*v*dx

In [ ]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [ ]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()

Redraw()

### Adjoint equation

We set up the adjoint equation
\begin{align*}
    \mbox{Find } p \in H_0^1(\Omega): \int_\Omega \nabla w \cdot \nabla p \, \mbox dx = - \partial_u J(u)(w) \quad \text{ for all } w \in H_0^1(\Omega)
\end{align*}
where $u$ is the solution to the state equation. For $J(u) = \int_\Omega |u-u_d|^q \mbox dx$, we get
\begin{align*}
    \partial_u J(u)(w) = q \int_\Omega (u-u_d)^{d-1}w \,\mbox dx.
\end{align*}
However, we can also use the Diff(...) command:

In [ ]:
q=6

def Cost(u): 
    return (u-ud)**q*dx

p, w = fes.TnT()
gfp = GridFunction(fes)
Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
fadjoint += -1*Cost(gfu).Diff(gfu,w)

In [ ]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [ ]:
fadjoint.Assemble()
SolveAdjointEquation()
Redraw()

Note that (for linear problems) the operator on the left hand side of the adjoint equation is just the transpose of the state operator.

### Automatic Shape Differentiation

The formula for the shape derivative was derived as the partial derivative of the perturbed Lagrangian (brought back to the original domain):
\begin{align*}
    d\mathcal J(\Omega; X) = \frac{\partial}{\partial t} \left( \int_\Omega \xi(t)|u - u_d^t|^q \mbox{d} x 
         +  \int_{\Omega} (F_t^{-\top}\nabla u) \cdot (F_t^{-\top} \nabla p) \xi(t) \, dx - \int_{\Omega} \xi(t) f^t p \,dx \right)|_{t=0} 
\end{align*}
where 
<ul>
    <li>   $T_t(x)=x+tX(x)=y$ 
    <li> $F_t = DT_t = I+t DX$
    <li> $\xi(t) = \mbox{det}(F_t)$
    <li>  $u_d^t = u_d \circ T_t$
    <li> $f^t = f \circ T_t$
</ul>

The integrand depends on the parameter $t$ only via $\xi(t)$, $F_t$ and $T_t$. Denoting the integrand by $G^{u,p}$, the derivative is given by
\begin{align*}
     d\mathcal J(\Omega; X) =& \frac{d G^{u,p}}{d \xi} \frac{d \xi}{d t} + \frac{d  G^{u,p}}{d F} \frac{d F}{d t} + \frac{d  G^{u,p}}{dy} \cdot \frac{d T_t}{dt} \\
     =& \frac{d G^{u,p}}{d \xi} \mbox{div}(X) + \frac{d  G^{u,p}}{d F} DX + \frac{d  G^{u,p}}{dy} \cdot X
\end{align*}

In [ ]:
J = Parameter(1)
F = Id(2)
#Finv = 2*Id(2) - F    # consistent linearization at F = I (avoids calling Inv() for speedup)

def Equation(u,v):
    return ( (Inv(F.trans)*grad(u))*(Inv(F.trans)*grad(v))-f*v)*J*dx
#    return ( (Finv.trans*grad(u))*(Finv.trans*grad(v))-f*v)*J*dx

def CostAuto(u): 
    return J*(u-ud)**q*dx

In [ ]:
VEC = H1(mesh, order=2, dim=2)
PHI, X = VEC.TnT()

Lagrangian = CostAuto(gfu) + Equation(gfu,gfp)
dJOmegaAuto = LinearForm(VEC)
dJOmegaAuto += Lagrangian.Diff(J, div(X))
dJOmegaAuto += Lagrangian.Diff(F, grad(X).trans)   # grad(X).trans is Jacobian
dJOmegaAuto += Lagrangian.Diff(x, X[0]) + Lagrangian.Diff(y, X[1])

In [ ]:
b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

gfX = GridFunction(VEC)

# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

In [ ]:
def SolveDeformationEquationAuto():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmegaAuto.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [ ]:
b.Assemble()
dJOmegaAuto.Assemble()
SolveDeformationEquationAuto()
Draw(-gfX, mesh, "gfX")

In [ ]:
print('Cost at initial design', Integrate (CostAuto(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

In [ ]:
mesh.SetDeformation(gfset)
a.Assemble()
fstate.Assemble()
SolveStateEquation()
print('Cost at new design', Integrate (CostAuto(gfu), mesh))

Equation can also be used to define the bilinear form. The following defines the same bilinear form as above:

In [ ]:
aAuto = BilinearForm(fes, symmetric=True)
aAuto += Equation(u,v)

Thus, the user has to enter the PDE (in its transformed form) only once.

Finally, let us again run the full algorithm:

In [ ]:
#reset to and solve for initial configuration
gfset.Set((0,0))
mesh.SetDeformation(gfset)
a.Assemble()
fstate.Assemble()
SolveStateEquation()

LineSearch = False

visoptions.vecfunction='gfset'
visoptions.scalfunction='state'

iter_max = 600
Jold = Integrate(CostAuto(gfu), mesh)
converged = False
for k in range(iter_max):
    print('cost at iteration', k, ': ', Jold)
    mesh.SetDeformation(gfset)
    
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmegaAuto.Assemble()
    SolveDeformationEquationAuto()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    
    Jnew = Integrate(CostAuto(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            input('a')
            scale = scale / 2
            
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            fstate.Assemble()
            SolveStateEquation()
            Jnew = Integrate(CostAuto(gfu), mesh)
    
    if converged==True:
        break
    Jold = Jnew

    Redraw(blocking=True)